### This solution is inspired by [Alexandru Papiu](https://www.kaggle.com/code/apapiu/regularized-linear-models), [Serigne](https://www.kaggle.com/code/serigne/stacked-regressions-top-4-on-leaderboard/notebook) and [Lavanya Shukla](https://www.kaggle.com/code/lavanyashukla01/how-i-made-top-0-3-on-a-kaggle-competition/notebook)

In [ ]:
# Essentials
import numpy as np
import pandas as pd
import datetime
import random

# Plots
import seaborn as sns
import matplotlib.pyplot as plt

# Models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.svm import SVR
from mlxtend.regressor import StackingCVRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

# Stats
from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

# Misc
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA

pd.set_option('display.max_columns', None)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Imports & EDA

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')
train_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

train = train_data
test = test_data

train_test_data = pd.concat([train, test],axis=0,ignore_index=True)
train_test_data.describe()

In [ ]:
# Missing ratio of each feature
missing_ratio = train_test_data.isnull().sum() / len(train_test_data)
missing_ratio = missing_ratio[missing_ratio.values>0]
missing_ratio_low = missing_ratio[missing_ratio.values<=0.1]
missing_ratio_high = missing_ratio[missing_ratio.values>0.1]

fig = plt.figure(figsize=(10,4),dpi=100)
plt.subplot(1,2,1)
sns.barplot(x=missing_ratio_low.index,y=missing_ratio_low.values)
plt.xticks(rotation=90)
plt.subplot(1,2,2)
sns.barplot(x=missing_ratio_high.index,y=missing_ratio_high.values)
plt.xticks(rotation=90);

In [ ]:
fig = plt.figure(figsize=(6,4),dpi=100)
sns.heatmap(train_test_data.corr(),cmap='viridis')

In [ ]:
plt.rcParams['figure.figsize'] = (12.0, 6.0)
prices = pd.DataFrame({"price":train["SalePrice"], "log(price + 1)":np.log1p(train["SalePrice"])})
prices.hist()

### Feature Engineering

In [ ]:
# Fill in missing rows
train['LotFrontage'] = train.groupby('Neighborhood')['LotFrontage'].transform(lambda val: val.fillna(val.mean()))
train['MiscFeature'] = train['MiscFeature'].replace(to_replace=['Gar2','Othr','TenC'],value='Other')
train['MiscFeature'] = train['MiscFeature'].fillna(value='Other')

test['LotFrontage'] = test.groupby('Neighborhood')['LotFrontage'].transform(lambda val: val.fillna(val.mean()))
test['MiscFeature'] = test['MiscFeature'].replace(to_replace=['Gar2','Othr','TenC'],value='Other')
test['MiscFeature'] = test['MiscFeature'].fillna(value='Other')

# Drop some features with high missing ratio
train = train.drop(['Alley','FireplaceQu','PoolQC','Fence'],axis=1)
test = test.drop(['Alley','FireplaceQu','PoolQC','Fence'],axis=1)

# Fill in missing rows
train['MSZoning'] = train['MSZoning'].fillna(value='RL')
train['Exterior1st'] = train['Exterior1st'].fillna(value='VinylSd')
train['Exterior2nd'] = train['Exterior2nd'].fillna(value='VinylSd')
train['MasVnrType'] = train['MasVnrType'].fillna(value='None')
train['MasVnrArea'] = train['MasVnrArea'].fillna(value=0)
train[['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2']] = train[['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2']].fillna(value='None')
train[['BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','BsmtFullBath','BsmtHalfBath']] = train[['BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','BsmtFullBath','BsmtHalfBath']].fillna(value=0)
train['Electrical'] = train['Electrical'].fillna(value='SBrkr')
train['KitchenQual'] = train['KitchenQual'].fillna(value='TA')
train['Functional'] = train['Functional'].fillna(value='Typ')
train[['GarageType','GarageFinish','GarageQual','GarageCond']] = train[['GarageType','GarageFinish','GarageQual','GarageCond']].fillna(value='None')
train[['GarageYrBlt','GarageCars','GarageArea']] = train[['GarageYrBlt','GarageCars','GarageArea']].fillna(value=0)
train['SaleType'] = train['SaleType'].fillna(value='WD')

test['MSZoning'] = test['MSZoning'].fillna(value='RL')
test['Exterior1st'] = test['Exterior1st'].fillna(value='VinylSd')
test['Exterior2nd'] = test['Exterior2nd'].fillna(value='VinylSd')
test['MasVnrType'] = test['MasVnrType'].fillna(value='None')
test['MasVnrArea'] = test['MasVnrArea'].fillna(value=0)
test[['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2']] = test[['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2']].fillna(value='None')
test[['BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','BsmtFullBath','BsmtHalfBath']] = test[['BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','BsmtFullBath','BsmtHalfBath']].fillna(value=0)
test['Electrical'] = test['Electrical'].fillna(value='SBrkr')
test['KitchenQual'] = test['KitchenQual'].fillna(value='TA')
test['Functional'] = test['Functional'].fillna(value='Typ')
test[['GarageType','GarageFinish','GarageQual','GarageCond']] = test[['GarageType','GarageFinish','GarageQual','GarageCond']].fillna(value='None')
test[['GarageYrBlt','GarageCars','GarageArea']] = test[['GarageYrBlt','GarageCars','GarageArea']].fillna(value=0)
test['SaleType'] = test['SaleType'].fillna(value='WD')

# Drop some features
train = train.drop(['Utilities','Id'],axis=1)
test = test.drop(['Utilities','Id'],axis=1)

In [ ]:
# Data Scaling

# Log transform the target
train['SalePrice'] = np.log1p(train['SalePrice'])

# Log/BoxCox transform skewed numeric features
numeric_feats = test.dtypes[test.dtypes != "object"].index

skewed_feats = test[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

# train_test_data[skewed_feats] = np.log1p(train_test_data[skewed_feats])
lam = 0.15
for feat in skewed_feats:
    train[feat] = boxcox1p(train[feat], lam)
    test[feat] = boxcox1p(test[feat], lam)

In [ ]:
y = train['SalePrice']
train = train.drop('SalePrice',axis=1)

train_test_data = pd.concat([train, test],axis=0,ignore_index=True)
train_test_data = pd.get_dummies(train_test_data,drop_first=True)

X = train_test_data.iloc[:1460]
X_submission = train_test_data.iloc[1460:]

### Modeling

In [ ]:
# Setup cross validation folds
kf = KFold(n_splits=12, random_state=42, shuffle=True)

# Define error metrics
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def cv_rmse(model, X=X):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kf))
    return (rmse)

In [ ]:
# Light Gradient Boosting Regressor
lightgbm = LGBMRegressor(objective='regression', 
                       num_leaves=6,
                       learning_rate=0.01, 
                       n_estimators=7000,
                       max_bin=200, 
                       bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.2,
                       feature_fraction_seed=8,
                       min_sum_hessian_in_leaf = 11,
                       verbose=-1,
                       random_state=42)

# XGBoost Regressor
xgboost = XGBRegressor(learning_rate=0.01,
                       n_estimators=6000,
                       max_depth=4,
                       min_child_weight=0,
                       gamma=0.6,
                       subsample=0.7,
                       colsample_bytree=0.7,
                       objective='reg:squarederror',
                       nthread=-1,
                       scale_pos_weight=1,
                       seed=27,
                       reg_alpha=0.00006,
                       random_state=42)

# Ridge Regressor
ridge_alphas = [1e-15, 1e-10, 1e-8, 9e-4, 7e-4, 5e-4, 3e-4, 1e-4, 1e-3, 5e-2, 1e-2, 0.1, 0.3, 1, 3, 5, 10, 15, 18, 20, 30, 50, 75, 100]
ridge = make_pipeline(RobustScaler(), RidgeCV(alphas=ridge_alphas, cv=kf))

# Support Vector Regressor
svr = make_pipeline(RobustScaler(), SVR(C= 20, epsilon= 0.008, gamma=0.0003))

# Gradient Boosting Regressor
gbr = GradientBoostingRegressor(n_estimators=6000,
                                learning_rate=0.01,
                                max_depth=4,
                                max_features='sqrt',
                                min_samples_leaf=15,
                                min_samples_split=10,
                                loss='huber',
                                random_state=42)  

# Random Forest Regressor
rf = RandomForestRegressor(n_estimators=1200,
                          max_depth=15,
                          min_samples_split=5,
                          min_samples_leaf=5,
                          max_features=None,
                          oob_score=True,
                          random_state=42)

# Stack up all the models above, optimized using xgboost
stack_gen = StackingCVRegressor(regressors=(xgboost, lightgbm, svr, ridge, gbr, rf),
                                meta_regressor=xgboost,
                                use_features_in_secondary=True)

In [ ]:
# Get cross validation scores for each model
scores = {}

# score = cv_rmse(lightgbm)
# print("lightgbm: {:.4f} ({:.4f})".format(score.mean(), score.std()))
# scores['lgb'] = (score.mean(), score.std())

# score = cv_rmse(xgboost)
# print("xgboost: {:.4f} ({:.4f})".format(score.mean(), score.std()))
# scores['xgb'] = (score.mean(), score.std())

# score = cv_rmse(svr)
# print("SVR: {:.4f} ({:.4f})".format(score.mean(), score.std()))
# scores['svr'] = (score.mean(), score.std())

# score = cv_rmse(ridge)
# print("ridge: {:.4f} ({:.4f})".format(score.mean(), score.std()))
# scores['ridge'] = (score.mean(), score.std())

# score = cv_rmse(rf)
# print("rf: {:.4f} ({:.4f})".format(score.mean(), score.std()))
# scores['rf'] = (score.mean(), score.std())

# score = cv_rmse(gbr)
# print("gbr: {:.4f} ({:.4f})".format(score.mean(), score.std()))
# scores['gbr'] = (score.mean(), score.std())

In [ ]:
# Fit the models
print('stack_gen')
stack_gen_model = stack_gen.fit(np.array(X), np.array(y))

print('lightgbm')
lgb_model_full_data = lightgbm.fit(X, y)

print('xgboost')
xgb_model_full_data = xgboost.fit(X, y)

print('Svr')
svr_model_full_data = svr.fit(X, y)

print('Ridge')
ridge_model_full_data = ridge.fit(X, y)

print('RandomForest')
rf_model_full_data = rf.fit(X, y)

print('GradientBoosting')
gbr_model_full_data = gbr.fit(X, y)

In [ ]:
# Plot the scores for each model
# sns.set_style("white")
# fig = plt.figure(figsize=(10, 5))

# ax = sns.pointplot(x=list(scores.keys()), y=[score for score, _ in scores.values()], markers=['o'], linestyles=['-'])
# for i, score in enumerate(scores.values()):
#     ax.text(i, score[0] + 0.002, '{:.6f}'.format(score[0]), horizontalalignment='left', size='large', color='black', weight='semibold')

# plt.ylabel('Score (RMSE)', size=20, labelpad=12.5)
# plt.xlabel('Model', size=20, labelpad=12.5)
# plt.tick_params(axis='x', labelsize=13.5)
# plt.tick_params(axis='y', labelsize=12.5)

# plt.title('Scores of Models', size=20)

# plt.show()

In [ ]:
# Blend models in order to make the final predictions more robust to overfitting

# Submission V1: blended model without Stacking Regressor
# def blended_predictions(X):
#     return ((0.2 * ridge_model_full_data.predict(X)) + \
#             (0.2 * svr_model_full_data.predict(X)) + \
#             (0.2 * gbr_model_full_data.predict(X)) + \
#             (0.1 * xgb_model_full_data.predict(X)) + \
#             (0.2 * lgb_model_full_data.predict(X)) + \
#             (0.1 * rf_model_full_data.predict(X)))

# Submission V2: blended model with Stacking Regressor
# def blended_predictions(X):
#     return ((0.1 * ridge_model_full_data.predict(X)) + \
#             (0.2 * svr_model_full_data.predict(X)) + \
#             (0.1 * gbr_model_full_data.predict(X)) + \
#             (0.1 * xgb_model_full_data.predict(X)) + \
#             (0.1 * lgb_model_full_data.predict(X)) + \
#             (0.05 * rf_model_full_data.predict(X)) + \
#             (0.35 * stack_gen_model.predict(np.array(X))))

# Submission V3: blended model with Stacking Regressor
def blended_predictions(X):
    return ((0.1 * ridge_model_full_data.predict(X)) + \
            (0.1 * svr_model_full_data.predict(X)) + \
            (0.1 * gbr_model_full_data.predict(X)) + \
            (0.05 * xgb_model_full_data.predict(X)) + \
            (0.1 * lgb_model_full_data.predict(X)) + \
            (0.05 * rf_model_full_data.predict(X)) + \
            (0.50 * stack_gen_model.predict(np.array(X))))

blended_score = rmsle(y, blended_predictions(X))
scores['blended'] = (blended_score, 0)
print('RMSLE score on train data:')
print(blended_score)

In [ ]:
# Read in sample_submission dataframe
submission = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv")
submission.iloc[:,1] = np.floor(np.expm1(blended_predictions(X_submission)))

# Fix outleir predictions
q1 = submission['SalePrice'].quantile(0.0045)
q2 = submission['SalePrice'].quantile(0.99)
submission['SalePrice'] = submission['SalePrice'].apply(lambda x: x if x > q1 else x*0.77)
submission['SalePrice'] = submission['SalePrice'].apply(lambda x: x if x < q2 else x*1.1)

submission.to_csv("submission_v3.csv", index=False)